In [1]:
import numpy as np 
import pandas as pd
import os
import joblib
import sklearn 
import matplotlib
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

#Regressions:

from sklearn.multioutput import MultiOutputRegressor

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

#Metric
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score


from pandas import DataFrame


# Show progress bar
from tqdm import tqdm

In [2]:
df = pd.read_csv('flo_dataset_augmented.csv')
df

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,in_amount_mmol,p_amount_mmol,ligand_amount_mmol,first_sol_amount_ml,second_sol_amount_ml,third_sol_amount_ml,...,x6_zinc stearate,x6_zinc undecylenate,x7_None,x7_copper bromide,x7_oleic acid,x7_water,x7_zinc iodide,diameter_nm,abs_nm,emission_nm
0,0,0,0,0,0.154575,-0.011188,-0.247025,-0.673379,-0.370637,-0.096002,...,1,0,1,0,0,0,0,1.750000,480.0,539.0
1,1,1,1,1,0.195883,0.280681,-0.439228,-0.658075,-0.370637,-0.096002,...,0,0,1,0,0,0,0,2.610000,610.0,688.5
2,2,2,2,2,-0.423738,-0.412509,-0.439228,-0.655198,-0.370637,-0.096002,...,0,0,1,0,0,0,0,3.088889,560.0,595.0
3,3,3,3,3,-0.423738,-0.412509,-0.439228,-0.655198,-0.370637,-0.096002,...,0,0,1,0,0,0,0,3.233333,590.0,635.0
4,4,4,4,4,0.154575,-0.011188,-0.247025,-0.604513,-0.370637,-0.096002,...,0,0,1,0,0,0,0,2.383333,482.2,500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,210,214,214,214,0.154575,-0.376755,0.713993,4.491519,-0.370637,-0.096002,...,0,0,1,0,0,0,0,4.000000,585.0,630.0
211,211,215,215,215,7.590031,7.504453,-0.439228,5.524499,-0.370637,-0.096002,...,0,0,1,0,0,0,0,2.833333,465.0,550.0
212,212,216,216,216,2.633061,0.718486,4.173657,6.213152,4.521657,-0.096002,...,0,0,1,0,0,0,0,1.933333,465.0,595.0
213,213,217,217,217,2.633061,0.718486,4.173657,6.213152,4.521657,-0.096002,...,0,0,1,0,0,0,0,3.033333,495.0,595.0


In [3]:
# Input for ML models


input_col = ['in_amount_mmol', 'p_amount_mmol', 'ligand_amount_mmol',
       'first_sol_amount_ml', 'second_sol_amount_ml', 'third_sol_amount_ml',
       'other_1_amount_mmol', 'other_2_amount_mmol', 'total_volume_ml',
       'temp_c', 'time_min', 'x0_chloroindium oxalate', 'x0_indium acetate',
       'x0_indium bromide', 'x0_indium chloride', 'x0_indium iodide',
       'x0_indium myristate', 'x0_indium oxalate', 'x0_indium palmitate',
       'x0_indium trifluoroacetate',
       'x0_indium tris(N,N-diisopropylacetamidinato)',
       'x1_bis(trimethylsilyl)phosphine', 'x1_phosphine gas',
       'x1_phosphorus trichloride', 'x1_sodium phosphide',
       'x1_tris(diethylamino)phosphine', 'x1_tris(dimethylamino)phosphine',
       'x1_tris(trimethylgermyl)phosphine', 'x1_tris(trimethylsilyl)phosphine',
       'x1_white phosphorus', 'x2_None', 'x2_dodecanethiol', 'x2_lauric acid',
       'x2_myristic acid', 'x2_oleic acid', 'x2_palmitic acid',
       'x2_stearic acid', 'x3_4-ethylpyridine', 'x3_None',
       'x3_dimethylformamide', 'x3_dodecylamine', 'x3_mesitylene',
       'x3_octadecene', 'x3_oleylamine', 'x3_trioctylamine',
       'x3_trioctylphosphine', 'x3_trioctylphosphine oxide', 'x4_None',
       'x4_dioctyl ether', 'x4_dioctylamine', 'x4_hexadecylamine',
       'x4_octylamine', 'x4_oleylamine', 'x4_toluene', 'x4_trioctylphosphine',
       'x4_trioctylphosphine oxide', 'x5_None', 'x5_trioctylphosphine',
       'x6_None', 'x6_acetic acid', 'x6_superhydride',
       'x6_tetrabutylammonium myristate', 'x6_zinc acetate', 'x6_zinc bromide',
       'x6_zinc chloride', 'x6_zinc iodide', 'x6_zinc octanoate',
       'x6_zinc oleate', 'x6_zinc stearate', 'x6_zinc undecylenate', 'x7_None',
       'x7_copper bromide', 'x7_oleic acid', 'x7_water', 'x7_zinc iodide']

output_col = ['diameter_nm', 'abs_nm', 'emission_nm']

X = df[input_col]

Y = df[output_col]

# Random Sampling

In [16]:
sampling = ("6/94", "10/90", "15/85", "20/80", "25/75", "30/70", "40/60", "50/50")
test_size = (0.06, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5)
for i in range(0, 8):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size[i], random_state=45, shuffle=True)
    ET_regr = ExtraTreesRegressor(n_estimators=5,
                            max_features=5,     
                            random_state=13).fit(X_train, Y_train)
            
    ET_regr.fit(X_train, Y_train)            
    ET_Y_pred = ET_regr.predict(X_test)
    ET_MAE = mean_absolute_error(Y_test, pd.DataFrame(ET_Y_pred))
    print(sampling[i], ": ", ET_MAE)

6/94 :  10.586820512825641
10/90 :  9.40112121208788
15/85 :  9.173946127921214
20/80 :  15.484334194626356
25/75 :  16.68445267488642
30/70 :  16.018196011354874
40/60 :  21.15738329025426
50/50 :  20.393502743470986


In [19]:
sampling = ("6/94", "10/90", "15/85", "20/80", "25/75", "30/70", "40/60", "50/50")
test_size = (0.06, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5)
for i in range(0, 8):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size[i], random_state=45, shuffle=True)
    DT_regr = DecisionTreeRegressor(max_depth=10,
                                max_features=6,
                                random_state=10)
            
    DT_regr.fit(X_train, Y_train)            
    DT_Y_pred = DT_regr.predict(X_test)
    DT_MAE = mean_absolute_error(Y_test, pd.DataFrame(DT_Y_pred))
    print(sampling[i], ": ", DT_MAE)

6/94 :  16.68935490434158
10/90 :  19.802365319821345
15/85 :  19.54955871846859
20/80 :  20.724260921439967
25/75 :  21.7932087334183
30/70 :  20.53286693346811
40/60 :  24.93661165455025
50/50 :  28.133617969805044


# Stratified Sampling

In [21]:
from sklearn.model_selection import StratifiedShuffleSplit

df2 = df.sort_values(by = 'emission_nm')
bins = [450, 500, 550, 600, 650, 700, 800]
df2['binned'] = pd.cut(df2['emission_nm'], bins)
df2

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,in_amount_mmol,p_amount_mmol,ligand_amount_mmol,first_sol_amount_ml,second_sol_amount_ml,third_sol_amount_ml,...,x6_zinc undecylenate,x7_None,x7_copper bromide,x7_oleic acid,x7_water,x7_zinc iodide,diameter_nm,abs_nm,emission_nm,binned
183,183,187,187,187,-0.299814,0.426616,-0.266245,0.084140,1.097051,-0.096002,...,0,1,0,0,0,0,6.333333,405.0,470.0,"(450, 500]"
184,184,188,188,188,-0.299814,0.426616,0.079721,0.084140,1.097051,-0.096002,...,0,1,0,0,0,0,6.333333,416.0,475.0,"(450, 500]"
95,95,95,95,95,-0.506354,-0.594927,-0.170143,-0.225754,-0.273281,-0.096002,...,0,1,0,0,0,0,3.033333,408.0,475.0,"(450, 500]"
168,168,172,172,172,-0.175889,-0.448993,0.252705,0.015274,0.118592,-0.096002,...,0,1,0,0,0,0,2.000000,430.0,477.0,"(450, 500]"
167,167,171,171,171,-0.175889,-0.448993,0.252705,0.015274,0.118592,-0.096002,...,0,0,0,1,0,0,2.000000,430.0,477.0,"(450, 500]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,19,19,19,19,-0.588970,1.448160,-0.323906,-0.535648,-0.370637,-0.096002,...,0,1,0,0,0,0,2.500000,601.0,695.0,"(650, 700]"
27,27,27,27,27,0.782458,0.017999,-0.439228,-0.508102,-0.261920,-0.096002,...,0,1,0,0,0,0,4.600000,650.0,709.0,"(700, 800]"
176,176,180,180,180,1.253370,1.448160,-0.439228,0.084140,0.118592,-0.096002,...,0,1,0,0,0,0,4.200000,626.0,729.0,"(700, 800]"
177,177,181,181,181,1.253370,1.448160,-0.439228,0.084140,0.118592,-0.096002,...,0,1,0,0,0,0,4.800000,689.0,765.0,"(700, 800]"


In [23]:
sampling = ("6/94", "10/90", "15/85", "20/80", "25/75", "30/70", "40/60", "50/50")
test_size = (0.06, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5)
for i in range(0, 8):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size[i], random_state=45, shuffle=True, stratify=df2['binned'])
    ET_regr = ExtraTreesRegressor(n_estimators=5,
                            max_features=5,     
                            random_state=13).fit(X_train, Y_train)
            
    ET_regr.fit(X_train, Y_train)            
    ET_Y_pred = ET_regr.predict(X_test)
    ET_MAE = mean_absolute_error(Y_test, pd.DataFrame(ET_Y_pred))
    print(ET_MAE)

18.08275783474872
17.195430976419196
18.421400673385858
20.53632902668372
18.808139917679014
17.2548102564
18.015645133486046
22.263482167345682


In [27]:
sampling = ("6/94", "10/90", "15/85", "20/80", "25/75", "30/70", "40/60", "50/50")
test_size = (0.06, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5)
for i in range(0, 8):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size[i], random_state=45, shuffle=True, stratify=df2['binned'])
    DT_regr = DecisionTreeRegressor(max_depth=10,
                                max_features=6,
                                random_state=10)
            
    DT_regr.fit(X_train, Y_train)            
    DT_Y_pred = DT_regr.predict(X_test)
    DT_MAE = mean_absolute_error(Y_test, pd.DataFrame(DT_Y_pred))
    print(DT_MAE)

32.11454619453993
26.34835424954062
23.601569397649413
26.978202165420154
27.493630380791558
25.368392190088517
21.626971270370916
24.145218188387076
